In [1]:
import xarray as xr
import s3fs
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset
from matplotlib.colors import LogNorm
import matplotlib.path as mpath
from functions import compute_ivt,to_nc
from IPython import display

In [2]:
def read_pangeo(start_year, end_year, experimentid):
    cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
    col = intake.open_esm_datastore(cat_url)
    cat = col.search(source_id=['NorESM2-LM'], experiment_id=['experimentid'], table_id=['day'], variable_id=['pr','clt','hus','va','tas'], member_id=['r1i1p1f1'])
    dset_dict = cat.to_dataset_dict(zarr_kwargs={'use_cftime':True})
    dataset_list = list(dset_dict.keys())
    start_year_ = start_year
    end_year_ = end_year
    df = dset_dict[dataset_list[0]]
    df_sliced = cm245.sel(time = slice(str(start_year)+"-01-01", str(end_year)+"-12-31"))
    return df_sliced

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,ScenarioMIP,NCC,NorESM2-LM,ssp245,r1i1p1f1,day,va,gn,gs://cmip6/CMIP6/ScenarioMIP/NCC/NorESM2-LM/ss...,NaN,20191108
1,ScenarioMIP,NCC,NorESM2-LM,ssp245,r1i1p1f1,day,tas,gn,gs://cmip6/CMIP6/ScenarioMIP/NCC/NorESM2-LM/ss...,NaN,20191108
2,ScenarioMIP,NCC,NorESM2-LM,ssp245,r1i1p1f1,day,pr,gn,gs://cmip6/CMIP6/ScenarioMIP/NCC/NorESM2-LM/ss...,NaN,20191108
3,ScenarioMIP,NCC,NorESM2-LM,ssp245,r1i1p1f1,day,hus,gn,gs://cmip6/CMIP6/ScenarioMIP/NCC/NorESM2-LM/ss...,NaN,20191108
4,ScenarioMIP,NCC,NorESM2-LM,ssp245,r1i1p1f1,day,clt,gn,gs://cmip6/CMIP6/ScenarioMIP/NCC/NorESM2-LM/ss...,NaN,20191108



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [3]:
# import AOD
s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", 
                       secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0",
                       client_kwargs=dict(endpoint_url="https://rgw.met.no"))
s3.ls('escience2022/Remy/')

['escience2022/Remy/.keep',
 'escience2022/Remy/.wget-20221102193645.sh.status',
 'escience2022/Remy/.wget-20221102193729.sh.status',
 'escience2022/Remy/.wget-20221102193735.sh.status',
 'escience2022/Remy/.wget-20221102193742.sh.status',
 'escience2022/Remy/Era5',
 'escience2022/Remy/dl_era5_hus.py',
 'escience2022/Remy/dl_era5_wind.py',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20150101-20201231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20210101-20301231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20310101-20401231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20410101-20501231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20510101-20601231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20610101-20701231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20710101-20801231.nc',
 'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20810101-20901231.nc',
 'escience

In [17]:
# import AOD ssp245 for 2035-2050
s3path = list([
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_ssp245_r1i1p1f1_gn_20310101-20401231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_ssp245_r1i1p1f1_gn_20410101-20501231.nc'
])

sopenlist=[s3.open(ss) for ss in s3path]

aod245_3550 = (xr.open_mfdataset(sopenlist)).sel(time = slice(str(start_year)+"-01-01", str(end_year)+"-12-31"))

In [ ]:
s3path = list([
    'escience2022/Remy/od550aer_AERday_NorESM2-LM_ssp370_r1i1p1f1_gn_20810101-20901231.nc',
    'escience2022/Remy/od550aer_AERday_NorESM2-LM_ssp370_r1i1p1f1_gn_20910101-21001231.nc'
])
sopenlist=[s3.open(ss) for ss in s3path]
aod370= (xr.open_mfdataset(sopenlist)).sel(time = slice(str(start_year2)+"-01-01", str(end_year2)+"-12-31"))